# 1. Data Preprocess

In [1]:
import pandas as pd

store_sales = pd.read_csv(
    'data/train.csv',
    usecols=['store_nbr', 'family', 'date', 'sales', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'sales': 'float32',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
)
store_sales['date'] = store_sales.date.dt.to_period('D')
store_sales = store_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

In [2]:
test_sales = pd.read_csv(
    'data/test.csv',
    usecols=['store_nbr', 'family', 'date', 'onpromotion'],
    dtype={
        'store_nbr': 'category',
        'family': 'category',
        'onpromotion': 'uint32',
    },
    parse_dates=['date'],
)
test_sales['date'] = test_sales.date.dt.to_period('D')
test_sales = test_sales.set_index(['store_nbr', 'family', 'date']).sort_index()

In [3]:
holidays_events = pd.read_csv(
    'data/holidays_events.csv',
    dtype={
        'type': 'category',
        'locale': 'category',
        'locale_name': 'category',
        'description': 'category',
        'transferred': 'bool',
    },
    parse_dates=['date'],
)
holidays_events = holidays_events.set_index('date').to_period('D')

# 2. Feature Engineering

In [4]:
def make_shifts(ts, begin, end):
    return pd.concat(
        {
            f'y_shift_{i}': ts.shift(i)
            for i in range(begin, end+1)
        },
        axis=1,
    )

y = store_sales.loc[:, 'sales'].unstack(['store_nbr', 'family'])
X_lag = make_shifts(y, 1, 1).dropna().loc['2017':]
y = y.loc['2017':]
display(X_lag)

y_shift_1                                                \
store_nbr           1                                                 
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01        2.0       0.0    3.0    2318.0   1.0   334.964996   
2017-01-02        0.0       0.0    0.0       0.0   0.0     0.000000   
2017-01-03        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-04        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-05        1.0       0.0    4.0    3039.0   2.0   543.250977   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        9.0       0.0   10.0    2377.0   0.0   417.020996   
2017-08-12        1.0       0.0    1.0    1006.0   0.0   145.606995   
2017-08-13        6.0       0.0    3.0    1659.0   0.0   243.220001   
2017-08-14        1.0       0.0    1.0     803.0   0.0   136.679001   
2017-08-15        1.0       0.0    6.0    2201.0   0.0   346.037994   

                                                     ...            \
store_nbr                                            ...         9   
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-01-01        10.0    518.0   631.0  133.266998  ...       3.0   
2017-01-02         0.0      0.0     0.0    0.000000  ...       0.0   
2017-01-03         0.0    332.0   376.0   44.980000  ...       5.0   
2017-01-04        15.0    952.0  1045.0  209.300003  ...       2.0   
2017-01-05        17.0   1055.0  1029.0  135.944000  ...       3.0   
...                ...      ...     ...         ...  ...       ...   
2017-08-11        17.0    696.0   736.0  139.761002  ...       3.0   
2017-08-12         4.0    341.0   343.0   64.302002  ...       5.0   
2017-08-13         3.0    351.0   526.0   99.487999  ...       2.0   
2017-08-14         1.0    169.0   266.0   47.770000  ...       3.0   
2017-08-15         4.0    571.0   699.0  154.578003  ...      12.0   

                                                                           \
store_nbr                                                                   
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01  278.092010         516.0         18.0                     4.0   
2017-01-02    0.000000           0.0          0.0                     0.0   
2017-01-03  659.570007        1243.0         11.0                    41.0   
2017-01-04  547.364014         876.0          6.0                    15.0   
2017-01-05  395.287994         677.0          6.0                    13.0   
...                ...           ...          ...                     ...   
2017-08-11  484.434021         329.0         11.0                     4.0   
2017-08-12  309.244995         373.0         11.0                     2.0   
2017-08-13  260.298004         400.0          7.0                    10.0   
2017-08-14  327.205994         510.0          2.0                     9.0   
2017-08-15  330.975006         445.0          2.0                    14.0   

                                                                               \
store_nbr                                                                       
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01  687.853027     100.404999  3091.355957                        2.0   
2017-01-02    0.000000       0.000000     0.000000                        0.0   
2017-01-03  843.596008     115.188995  3136.895996                        1.0   
2017-01-04  714.659973     133.039001  3229.558105                        1.0   
2017-01-05  536.830017      75.201004  1491.416992                        7.0   
...       

In [5]:
X_lag.columns

MultiIndex([('y_shift_1', '1',                 'AUTOMOTIVE'),
            ('y_shift_1', '1',                  'BABY CARE'),
            ('y_shift_1', '1',                     'BEAUTY'),
            ('y_shift_1', '1',                  'BEVERAGES'),
            ('y_shift_1', '1',                      'BOOKS'),
            ('y_shift_1', '1',               'BREAD/BAKERY'),
            ('y_shift_1', '1',                'CELEBRATION'),
            ('y_shift_1', '1',                   'CLEANING'),
            ('y_shift_1', '1',                      'DAIRY'),
            ('y_shift_1', '1',                       'DELI'),
            ...
            ('y_shift_1', '9',                  'MAGAZINES'),
            ('y_shift_1', '9',                      'MEATS'),
            ('y_shift_1', '9',              'PERSONAL CARE'),
            ('y_shift_1', '9',               'PET SUPPLIES'),
            ('y_shift_1', '9',    'PLAYERS AND ELECTRONICS'),
            ('y_shift_1', '9',                    'POU

In [6]:
holidays = (
    holidays_events
    .query("locale in ['National']")
    .loc['2017':'2017-08-31', ['description']]
    .assign(description=lambda x: x.description.cat.remove_unused_categories())
)
display(holidays)
X_holidays = pd.get_dummies(holidays, dtype=float)
X_holidays = X_holidays.groupby(X_holidays.index).sum()

X_holidays, X_holidays_test = X_holidays.loc[:'2017-08-15'], X_holidays.loc['2017-08-16':]
display(X_holidays)
display(X_holidays_test)

,description
date,
2017-01-01,Primer dia del ano
2017-01-02,Traslado Primer dia del ano
2017-02-27,Carnaval
2017-02-28,Carnaval
2017-04-14,Viernes Santo
2017-05-01,Dia del Trabajo
2017-05-13,Dia de la Madre-1
2017-05-14,Dia de la Madre
2017-05-24,Batalla de Pichincha


,description_Batalla de Pichincha,description_Carnaval,description_Dia de la Madre,description_Dia de la Madre-1,description_Dia del Trabajo,description_Primer Grito de Independencia,description_Primer dia del ano,description_Traslado Batalla de Pichincha,description_Traslado Primer Grito de Independencia,description_Traslado Primer dia del ano,description_Viernes Santo
date,,,,,,,,,,,
2017-01-01,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2017-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2017-02-27,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-02-28,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-04-14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2017-05-01,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-13,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-14,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2017-05-24,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,description_Batalla de Pichincha,description_Carnaval,description_Dia de la Madre,description_Dia de la Madre-1,description_Dia del Trabajo,description_Primer Grito de Independencia,description_Primer dia del ano,description_Traslado Batalla de Pichincha,description_Traslado Primer Grito de Independencia,description_Traslado Primer dia del ano,description_Viernes Santo
date,,,,,,,,,,,


## Promotion

In [7]:
X_promotion = store_sales.loc[:, ['onpromotion']].copy()
X_promotion = X_promotion.unstack(['store_nbr', 'family']).loc['2017':]

X_promotion_test = test_sales.loc[:, ['onpromotion']].copy()
X_promotion_test = X_promotion_test.unstack(['store_nbr', 'family'])

display(X_promotion)
display(X_promotion_test)

onpromotion                                                \
store_nbr            1                                                 
family      AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                   
2017-01-01           0         0      0         0     0            0   
2017-01-02           0         0      0        31     0            2   
2017-01-03           0         0      1        42     0            2   
2017-01-04           0         0      1        54     0            8   
2017-01-05           0         0      2        32     0            7   
...                ...       ...    ...       ...   ...          ...   
2017-08-11           0         0      1         3     0            8   
2017-08-12           0         0      1         7     0            6   
2017-08-13           0         0      0         5     0            6   
2017-08-14           0         0      1         9     0            9   
2017-08-15           0         0      1        11     0            8   

                                            ...                                \
store_nbr                                   ...         9                       
family     CELEBRATION CLEANING DAIRY DELI  ... MAGAZINES MEATS PERSONAL CARE   
date                                        ...                                 
2017-01-01           0        0     0    0  ...         0     0             0   
2017-01-02           0        7    11    3  ...         0     0            13   
2017-01-03           0       18    14    5  ...         0     0            11   
2017-01-04           1       15    32    5  ...         0     0            15   
2017-01-05           0       10    24    1  ...         0    21             8   
...                ...      ...   ...  ...  ...       ...   ...           ...   
2017-08-11           0        7    14   35  ...         0     0            11   
2017-08-12           0        8    19    3  ...         0     0             7   
2017-08-13           0        5    15    4  ...         0     0             9   
2017-08-14           0       11    23    7  ...         0     0            10   
2017-08-15           0       10    19    5  ...         0     0            11   

                                                                        \
store_nbr                                                                
family     PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY PREPARED FOODS   
date                                                                     
2017-01-01            0                       0       0              0   
2017-01-02            0                       0       2              1   
2017-01-03            0                       0       1              2   
2017-01-04            0                       0       1              8   
2017-01-05            0                       0       1              1   
...                 ...                     ...     ...            ...   
2017-08-11            0                       0      22              3   
2017-08-12            0                       0       0              1   
2017-08-13            0                       0       0              1   
2017-08-14            0                       0       0              0   
2017-08-15            0                       0       0              1   

                                                       
store_nbr                                              
family     PRODUCE SCHOOL AND OFFICE SUPPLIES SEAFOOD  
date                                                   
2017-01-01       0                          0       0  
2017-01-02       4                          0       0  
2017-01-03     150                          0       0  
2017-01-04       9                          0       0  
2017-01-05       5                          0       1  
...            ...                        ...     ...  
2017-08-11       6                         

onpromotion                                                \
store_nbr            1                                                 
family      AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                   
2017-08-16           0         0      2        20     0           12   
2017-08-17           0         0      1        17     0            7   
2017-08-18           0         0      1        12     0           10   
2017-08-19           0         0      1        11     0            9   
2017-08-20           0         0      1        10     0            5   
2017-08-21           0         0      1        14     0           10   
2017-08-22           0         0      0         9     0            6   
2017-08-23           0         0      1        27     0           14   
2017-08-24           0         0      0        26     0            1   
2017-08-25           0         0      0        32     0            2   
2017-08-26           0         0      1        32     0            3   
2017-08-27           0         0      1        23     0            5   
2017-08-28           0         0      1        31     0            1   
2017-08-29           0         0      1        31     0            5   
2017-08-30           0         0      0        35     0            7   
2017-08-31           0         0      1        33     0            4   

                                            ...                                \
store_nbr                                   ...         9                       
family     CELEBRATION CLEANING DAIRY DELI  ... MAGAZINES MEATS PERSONAL CARE   
date                                        ...                                 
2017-08-16           0       25    45   18  ...         0     0            19   
2017-08-17           0        7    13    7  ...         0    21            10   
2017-08-18           0        6    18   52  ...         0     1             9   
2017-08-19           0       11    20    6  ...         0     1            11   
2017-08-20           0        4    17    6  ...         0     0            12   
2017-08-21           0        9    20    7  ...         0     0            13   
2017-08-22           0       10    18    7  ...         0     0            13   
2017-08-23           0       11    22   11  ...         0     0            15   
2017-08-24           0       12    13   11  ...         0    20            13   
2017-08-25           0        8    12   52  ...         0     0             9   
2017-08-26           0       10    12    8  ...         0     0            13   
2017-08-27           0        7     7    6  ...         0     0            14   
2017-08-28           0       11    12   10  ...         0     0            11   
2017-08-29           0       13    13    7  ...         0     0            12   
2017-08-30           0       11    16    5  ...         0     0            12   
2017-08-31           0       12    11    5  ...         0    20            14   

                                                                        \
store_nbr                                                                
family     PET SUPPLIES PLAYERS AND ELECTRONICS POULTRY PREPARED FOODS   
date                                                                     
2017-08-16            0                       0       0              1   
2017-08-17            0                       0       0              0   
2017-08-18            0                       0      14              2   
2017-08-19            0                       0       0              0   
2017-08-20            0                       0       1              0   
2017-08-21            0                       0       0              0   
2017-08-22            0                       0       2              0   
2017-08-23            0                       0       0              0   
2017-08-24            0                       0       0      

## Trend and Season

In [8]:
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
fourier = CalendarFourier(freq='ME', order=4)
dp = DeterministicProcess(
    y.index,
    constant=True,
    order=1,
    seasonal=True,
    additional_terms=[fourier],
    drop=True,
)
X_trend_season = dp.in_sample()
X_trend_season['NewYear'] = (X_trend_season.index.dayofyear == 1)

X_trend_season_test = dp.out_of_sample(16)
X_trend_season_test['NewYear'] = (X_trend_season_test.index.dayofyear == 1)

In [9]:
display(X_trend_season)
display(X_trend_season_test)

,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=ME)","cos(1,freq=ME)","sin(2,freq=ME)","cos(2,freq=ME)","sin(3,freq=ME)","cos(3,freq=ME)","sin(4,freq=ME)","cos(4,freq=ME)",NewYear
date,,,,,,,,,,,,,,,,,
2017-01-01,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,True
2017-01-02,1.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.201299,0.979530,0.394356,0.918958,0.571268,0.820763,0.724793,0.688967,False
2017-01-03,1.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.394356,0.918958,0.724793,0.688967,0.937752,0.347305,0.998717,-0.050649,False
2017-01-04,1.0,4.0,0.0,0.0,1.0,0.0,0.0,0.0,0.571268,0.820763,0.937752,0.347305,0.968077,-0.250653,0.651372,-0.758758,False
2017-01-05,1.0,5.0,0.0,0.0,0.0,1.0,0.0,0.0,0.724793,0.688967,0.998717,-0.050649,0.651372,-0.758758,-0.101168,-0.994869,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-08-11,1.0,223.0,0.0,0.0,0.0,0.0,1.0,0.0,0.897805,-0.440394,-0.790776,-0.612106,-0.201299,0.979530,0.968077,-0.250653,False
2017-08-12,1.0,224.0,0.0,0.0,0.0,0.0,0.0,1.0,0.790776,-0.612106,-0.968077,-0.250653,0.394356,0.918958,0.485302,-0.874347,False
2017-08-13,1.0,225.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651372,-0.758758,-0.988468,0.151428,0.848644,0.528964,-0.299363,-0.954139,False


,const,trend,"s(2,7)","s(3,7)","s(4,7)","s(5,7)","s(6,7)","s(7,7)","sin(1,freq=ME)","cos(1,freq=ME)","sin(2,freq=ME)","cos(2,freq=ME)","sin(3,freq=ME)","cos(3,freq=ME)","sin(4,freq=ME)","cos(4,freq=ME)",NewYear
2017-08-16,1.0,228.0,0.0,0.0,1.0,0.0,0.0,0.0,0.101168,-0.994869,-0.201299,0.979530,0.299363,-0.954139,-0.394356,0.918958,False
2017-08-17,1.0,229.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.101168,-0.994869,0.201299,0.979530,-0.299363,-0.954139,0.394356,0.918958,False
2017-08-18,1.0,230.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.299363,-0.954139,0.571268,0.820763,-0.790776,-0.612106,0.937752,0.347305,False
2017-08-19,1.0,231.0,0.0,0.0,0.0,0.0,0.0,1.0,-0.485302,-0.874347,0.848644,0.528964,-0.998717,-0.050649,0.897805,-0.440394,False
2017-08-20,1.0,232.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.651372,-0.758758,0.988468,0.151428,-0.848644,0.528964,0.299363,-0.954139,False
2017-08-21,1.0,233.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.790776,-0.612106,0.968077,-0.250653,-0.394356,0.918958,-0.485302,-0.874347,False
2017-08-22,1.0,234.0,0.0,1.0,0.0,0.0,0.0,0.0,-0.897805,-0.440394,0.790776,-0.612106,0.201299,0.979530,-0.968077,-0.250653,False
2017-08-23,1.0,235.0,0.0,0.0,1.0,0.0,0.0,0.0,-0.968077,-0.250653,0.485302,-0.874347,0.724793,0.688967,-0.848644,0.528964,False
2017-08-24,1.0,236.0,0.0,0.0,0.0,1.0,0.0,0.0,-0.998717,-0.050649,0.101168,-0.994869,0.988468,0.151428,-0.201299,0.979530,False
2017-08-25,1.0,237.0,0.0,0.0,0.0,0.0,1.0,0.0,-0.988468,0.151428,-0.299363,-0.954139,0.897805,-0.440394,0.571268,0.820763,False


# 3. Model Training

In [10]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

store_numbers = store_sales.index.get_level_values('store_nbr').unique()
families = store_sales.index.get_level_values('family').unique()
y_pred = []
for STORE_NBR in store_numbers:
    for FAMILY in families:
        idx = (slice(None), STORE_NBR, FAMILY)

        # Without lag
        X_i = pd.concat([X_promotion.loc[:, idx], X_trend_season], axis=1)
        X_i.columns = X_i.columns.astype(str)
        X_i = X_i.join(X_holidays, on='date').fillna(0.0)

        y_i = y.loc[:, idx[1:]]

        model = LinearRegression(fit_intercept=False)
        model.fit(X_i, y_i)

        # y_pred_i = []
        #
        # X_lag_pred = X_lag.loc[:, idx].iloc[[-1]]
        #
        # for i in range(16):
        #     t_i = X_trend_season_test.index[i]
        #
        #     X_lag_pred.index = [t_i]
        #     # X_lag_pred.iloc(axis=1)[1:] = X_lag_pred.iloc(axis=1)[:3]
        #     if i == 0:
        #         X_lag_pred.iloc(axis=1)[0] = y_i.iloc[-1]
        #     else:
        #         X_lag_pred.iloc(axis=1)[0] = pd.DataFrame([y_pred_i[i - 1]], dtype='float32').iloc[0]
        #
        #     X_pred_step_i = pd.concat([
        #         X_promotion_test.loc[[t_i], idx],
        #         X_trend_season_test.loc[[t_i]],
        #         X_lag_pred,
        #     ], axis=1)
        #     X_pred_step_i.columns = X_pred_step_i.columns.astype(str)
        #     X_pred_step_i.index.name = 'date'
        #     X_pred_step_i = X_pred_step_i.join(X_holidays_test, on='date').fillna(0.0)
        #
        #     y_pred_i.append(model.predict(X_pred_step_i).item())
        #
        # display(y_pred_i)

        X_pred_i = pd.concat([X_promotion_test.loc[:, idx], X_trend_season_test], axis=1)
        X_pred_i.columns = X_pred_i.columns.astype(str)
        X_pred_i.index.name = 'date'
        X_pred_i = X_pred_i.join(X_holidays_test, on='date').fillna(0.0)

        y_pred_i = pd.Series(model.predict(X_pred_i), index=X_promotion_test.index, name=y_i.name, dtype=y_i.dtype)
        y_pred.append(y_pred_i)

y_pred = pd.concat(y_pred, axis=1)

## 3.2 Without Lag

In [11]:
display(y_pred)
display(y)

1                                                          \
           AUTOMOTIVE BABY CARE    BEAUTY    BEVERAGES     BOOKS BREAD/BAKERY   
date                                                                            
2017-08-16   4.326180       0.0  6.386703  2313.101807  0.428136   392.971497   
2017-08-17   4.117145       0.0  3.834897  1986.116821  0.084033   338.173248   
2017-08-18   5.811297       0.0  3.838559  2279.241943  0.066917   370.492706   
2017-08-19   4.432638       0.0  4.274302  2313.163086 -0.244439   371.207397   
2017-08-20   1.690335       0.0  2.863219   932.889709 -0.264230   125.280243   
2017-08-21   4.139060       0.0  4.623882  2262.409180 -0.193614   374.168915   
2017-08-22   4.473398       0.0  2.855759  2209.066162  0.161942   337.458160   
2017-08-23   5.348356       0.0  4.984080  2448.504639  0.287913   400.096497   
2017-08-24   5.879647       0.0  2.565110  2095.576660  0.044625   333.750916   
2017-08-25   7.947556       0.0  2.496439  2419.820068  0.073445   356.595581   
2017-08-26   6.431101       0.0  4.591912  2376.355225 -0.253978   351.054504   
2017-08-27   3.067778       0.0  2.811822   844.035278 -0.333451   103.583580   
2017-08-28   4.621030       0.0  4.251179  2174.027832 -0.328697   352.184906   
2017-08-29   4.101851       0.0  4.276550  2194.577148 -0.004627   324.851135   
2017-08-30   4.463284       0.0  2.664859  2399.915039  0.149522   398.047516   
2017-08-31   4.977252       0.0  4.410511  2187.397949 -0.007819   348.299988   

                                                            ...         9  \
           CELEBRATION    CLEANING       DAIRY        DELI  ... MAGAZINES   
date                                                        ...             
2017-08-16   17.369678  813.339966  849.027100  155.107300  ...  3.773968   
2017-08-17   20.327396  617.901794  653.278259  118.201653  ...  3.311926   
2017-08-18   22.603821  695.195251  769.826050  175.846237  ...  3.362999   
2017-08-19   12.738450  567.816284  737.093567  139.747528  ...  4.176259   
2017-08-20    2.105043  167.959305  279.426697   60.512714  ...  5.241766   
2017-08-21   13.258381  618.843811  708.024292  145.485413  ...  2.971648   
2017-08-22   15.482290  714.206604  674.723206  132.595322  ...  2.849851   
2017-08-23   17.412470  797.562866  806.942017  139.746719  ...  2.830662   
2017-08-24   21.973421  634.563171  648.780334  116.910172  ...  2.466366   
2017-08-25   25.275282  688.854675  745.893433  165.976715  ...  2.469338   
2017-08-26   15.500068  532.273987  697.817322  132.052322  ...  3.204649   
2017-08-27    3.997435  118.573730  224.886673   50.666672  ...  4.294162   
2017-08-28   13.670411  554.798950  652.162354  144.039307  ...  2.232453   
2017-08-29   14.390957  659.864197  632.670715  133.119003  ...  2.493772   
2017-08-30   15.369878  768.287659  787.490173  139.844467  ...  2.927461   
2017-08-31   19.839937  656.154175  673.376099  126.436005  ...  2.923832   

                                                                           \
                 MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-08-16  314.938507    341.557617     7.638408                7.514024   
2017-08-17  533.229736    313.284332     6.530257                8.239716   
2017-08-18  304.606415    263.371002     7.227097                6.894063   
2017-08-19  414.066498    549.068176    10.207477               13.388874   
2017-08-20  407.435547    571.361938    10.040587               14.344654   
2017-08-21  305.335876    354.147339     5.752563                6.914579   
2017-08-22  286.320190    352.244354     5.230373                6.798695   
2017-08-23  275.076447    303.416962     6.848273                5.747468   
2017-08-24  477.630066    290.617889     5.858498                6.038357   
2017-08-25  253.391785    231.456833     6.883637                4.411037   
2017-08-26  369

store_nbr           1                                                \
family     AUTOMOTIVE BABY CARE BEAUTY BEVERAGES BOOKS BREAD/BAKERY   
date                                                                  
2017-01-01        0.0       0.0    0.0       0.0   0.0     0.000000   
2017-01-02        5.0       0.0    0.0    1434.0   0.0   166.819000   
2017-01-03        4.0       0.0    4.0    3081.0   2.0   519.348022   
2017-01-04        1.0       0.0    4.0    3039.0   2.0   543.250977   
2017-01-05        2.0       0.0    3.0    2617.0   0.0   533.479980   
...               ...       ...    ...       ...   ...          ...   
2017-08-11        1.0       0.0    1.0    1006.0   0.0   145.606995   
2017-08-12        6.0       0.0    3.0    1659.0   0.0   243.220001   
2017-08-13        1.0       0.0    1.0     803.0   0.0   136.679001   
2017-08-14        1.0       0.0    6.0    2201.0   0.0   346.037994   
2017-08-15        4.0       0.0    4.0    1942.0   0.0   329.541016   

store_nbr                                            ...         9  \
family     CELEBRATION CLEANING   DAIRY        DELI  ... MAGAZINES   
date                                                 ...             
2017-01-01         0.0      0.0     0.0    0.000000  ...       0.0   
2017-01-02         0.0    332.0   376.0   44.980000  ...       5.0   
2017-01-03        15.0    952.0  1045.0  209.300003  ...       2.0   
2017-01-04        17.0   1055.0  1029.0  135.944000  ...       3.0   
2017-01-05        40.0    918.0   853.0  137.005997  ...       2.0   
...                ...      ...     ...         ...  ...       ...   
2017-08-11         4.0    341.0   343.0   64.302002  ...       5.0   
2017-08-12         3.0    351.0   526.0   99.487999  ...       2.0   
2017-08-13         1.0    169.0   266.0   47.770000  ...       3.0   
2017-08-14         4.0    571.0   699.0  154.578003  ...      12.0   
2017-08-15        21.0    703.0   602.0  116.402000  ...      11.0   

store_nbr                                                                  \
family           MEATS PERSONAL CARE PET SUPPLIES PLAYERS AND ELECTRONICS   
date                                                                        
2017-01-01    0.000000           0.0          0.0                     0.0   
2017-01-02  659.570007        1243.0         11.0                    41.0   
2017-01-03  547.364014         876.0          6.0                    15.0   
2017-01-04  395.287994         677.0          6.0                    13.0   
2017-01-05  470.768005         604.0          7.0                    10.0   
...                ...           ...          ...                     ...   
2017-08-11  309.244995         373.0         11.0                     2.0   
2017-08-12  260.298004         400.0          7.0                    10.0   
2017-08-13  327.205994         510.0          2.0                     9.0   
2017-08-14  330.975006         445.0          2.0                    14.0   
2017-08-15  449.227997         522.0          6.0                     6.0   

store_nbr                                                                      \
family         POULTRY PREPARED FOODS      PRODUCE SCHOOL AND OFFICE SUPPLIES   
date                                                                            
2017-01-01    0.000000       0.000000     0.000000                        0.0   
2017-01-02  843.596008     115.188995  3136.895996                        1.0   
2017-01-03  714.659973     133.039001  3229.558105                        1.0   
2017-01-04  536.830017      75.201004  1491.416992                        7.0   
2017-01-05  414.100006     113.698997  1566.821045                        1.0   
...                ...            ...          ...                        ...   
2017-08-11  525.223999     112.099998  1453.078003                      140.0   
2017-08-12  383.386993     129.903992  1419.264038                      138.0   
2017-08-13  412.458008     105.168999  1693.607056    

In [12]:
y_pred_submit = y_pred.copy()
y_pred_submit = y_pred_submit.clip(lower=0)
y_pred_submit.columns.names = ['store_nbr', 'family']
y_pred_submit = y_pred_submit.stack(['store_nbr', 'family'], future_stack=True)
y_pred_submit = y_pred_submit.to_frame()
y_pred_submit.columns = pd.Index(['sales'])

In [13]:
csv_submit = y_pred_submit.reset_index().loc[:, ['sales']]
csv_submit.index = csv_submit.index + 3000888
csv_submit.index.names = ['id']
csv_submit.to_csv('results/submission.csv')
display(csv_submit)

,sales
id,
3000888,4.326180
3000889,0.000000
3000890,6.386703
3000891,2313.101807
3000892,0.428136
...,...
3029395,372.100403
3029396,105.162903
3029397,1200.498535


In [14]:
# print(store_sales.index.get_level_values('family').unique())

# ax = y_i.plot(figsize=(100, 8))
# ax = y_fit_i.plot(ax=ax)
# ax = y_pred_i.plot(ax=ax)
# ax = (y_i - y_fit_i).plot(figsize=(100, 8))
